In [4]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Log\training.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

target_col = 'kilometer_cont'
X = df.drop(target_col, axis=1)
y = df[target_col]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.60, random_state=42)

# Voting Regressor

In [14]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb


voting_estimators = [
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42)),
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('knn', KNeighborsRegressor()),
    ('xgb', xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42))
]

voting = VotingRegressor(estimators=voting_estimators)
voting.fit(X_train, y_train)
y_pred_voting = voting.predict(X_test)

print("Voting Regressor R2:", r2_score(y_test, y_pred_voting))
print("Voting Regressor MSE:", mean_squared_error(y_test, y_pred_voting))

Voting Regressor R2: 0.47526562168996256
Voting Regressor MSE: 5835713.515391274


# Stacking Regressor

In [15]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb

# Base estimators
stacking_estimators = [
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42)),
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('knn', KNeighborsRegressor()),
    ('xgb', xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42))
]

stacking = StackingRegressor(estimators=stacking_estimators, final_estimator=LinearRegression())
stacking.fit(X_train, y_train)
y_pred_stacking = stacking.predict(X_test)

print("Stacking Regressor R2:", r2_score(y_test, y_pred_stacking))
print("Stacking Regressor MSE:", mean_squared_error(y_test, y_pred_stacking))

Stacking Regressor R2: 0.5085176599501142
Stacking Regressor MSE: 5465908.568145334


# Bagging Regressor

In [10]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Bagging
bagging = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=50,
    random_state=42
)

bagging.fit(X_train, y_train)
y_pred_bagging = bagging.predict(X_test)

print("Bagging Regressor R2:", r2_score(y_test, y_pred_bagging))
print("Bagging Regressor MSE:", mean_squared_error(y_test, y_pred_bagging))

Bagging Regressor R2: 0.4941066793836645
Bagging Regressor MSE: 5628280.832292873


# Boosting Regressors

# Gradient Boosting

In [11]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)

print("Gradient Boosting R2:", r2_score(y_test, y_pred_gbr))
print("Gradient Boosting MSE:", mean_squared_error(y_test, y_pred_gbr))

Gradient Boosting R2: 0.5089259541774216
Gradient Boosting MSE: 5463410.024810046


# XGBoost

In [12]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost R2:", r2_score(y_test, y_pred_xgb))
print("XGBoost MSE:", mean_squared_error(y_test, y_pred_xgb))

XGBoost R2: 0.4805907076966903
XGBoost MSE: 5778651.832018529


| Model              | R²                                      |
| ------------------ | --------------------------------------- |
| Voting Regressor   | <span style="color:red">0.4753</span>   |
| Stacking Regressor | <span style="color:green">0.5085</span> |
| Bagging Regressor  | <span style="color:red">0.4941</span>   |
| Gradient Boosting  | <span style="color:green">0.5089</span> |
| XGBoost            | <span style="color:red">0.4806</span>   |


| Model              | MSE                                        |
| ------------------ | ------------------------------------------ |
| Voting Regressor   | <span style="color:red">5,835,713</span>   |
| Stacking Regressor | <span style="color:green">5,465,908</span> |
| Bagging Regressor  | <span style="color:red">5,628,280</span>   |
| Gradient Boosting  | <span style="color:green">5,463,410</span> |
| XGBoost            | <span style="color:red">5,778,651</span>   |
